# Test Valuation Engines

This notebook walks through:

1. Loading historical SOFR fixings
2. Building a simple two-index yield curve
3. Valuing various products (bullet, IBOR cashflow, OIS cashflow, futures, swaps)

---

## 1. Imports & Setup

In [1]:
import pandas as pd
from fixedincomelib.data import DataCollection, Data1D, build_yc_data_collection
from fixedincomelib.date import Date
from fixedincomelib.yield_curve import YieldCurve
from fixedincomelib.valuation import IndexManager
from fixedincomelib.valuation import ValuationEngineRegistry
from fixedincomelib.product import (
    ProductBulletCashflow,
    ProductIborCashflow,
    ProductOvernightIndexCashflow,
    ProductFuture,
    ProductRfrFuture,
    ProductIborSwap,
    ProductOvernightSwap,
    ProductPortfolio )
from fixedincomelib.builders import create_products_from_data1d
from fixedincomelib.builders import build_yc_calibration_basket
from fixedincomelib.product.product_display_visitor import RfrFutureVisitor, OvernightSwapVisitor

## 2. Load SOFR Fixings

In [2]:
IndexManager.instance()

## 3. Build Dummy Yield Curve

In [3]:
MARKET_DF = pd.DataFrame(
    [
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-09-24 x 2025-12-24", 95.70],
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-12-24 x 2026-03-24", 95.80],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-03-24 x 2026-06-24", 95.90],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-06-24 x 2026-09-24", 96.00],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-09-24 x 2026-12-24", 96.08],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-12-24 x 2027-03-24", 96.16],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-03-24 x 2027-06-24", 96.24],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-06-24 x 2027-09-24", 96.32],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-09-24 x 2027-12-24", 96.38],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-12-24 x 2028-03-24", 96.44], 
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-03-24 x 2028-06-24", 96.50],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-06-24 x 2028-09-24", 96.55],  
        ["RFR SWAP","USD-SOFR-OIS","4Y",  0.0368],
        ["RFR SWAP","USD-SOFR-OIS","5Y",  0.0365],
        ["RFR SWAP","USD-SOFR-OIS","6Y",  0.0371],
        ["RFR SWAP","USD-SOFR-OIS","7Y",  0.0374],
        ["RFR SWAP","USD-SOFR-OIS","8Y",  0.0380],
        ["RFR SWAP","USD-SOFR-OIS","9Y",  0.0383],
        ["RFR SWAP","USD-SOFR-OIS","10Y", 0.0386],  
        ["RFR SWAP","USD-SOFR-OIS","15Y", 0.0395],
        ["RFR SWAP","USD-SOFR-OIS","20Y", 0.0405],
        ["RFR SWAP","USD-SOFR-OIS","25Y", 0.0412],
        ["RFR SWAP","USD-SOFR-OIS","30Y", 0.0419],  
        ["RFR SWAP","USD-SOFR-OIS","40Y", 0.0423],
        ["RFR SWAP","USD-SOFR-OIS","50Y", 0.0428],
        ["RFR SWAP","USD-SOFR-OIS","60Y", 0.0432],
    ],
    columns=["DATA TYPE","DATA CONVENTION","AXIS","VALUE"],
)


In [4]:
IBOR_FUTURES_DF = pd.DataFrame(
    [
        ["IBOR FUTURE","USD-LIBOR-3M","2025-09-24", 95.40],
        ["IBOR FUTURE","USD-LIBOR-3M","2025-12-24", 95.50],
        ["IBOR FUTURE","USD-LIBOR-3M","2026-03-24", 95.60],
        ["IBOR FUTURE","USD-LIBOR-3M","2026-06-24", 95.70],
        ["IBOR FUTURE","USD-LIBOR-3M","2026-09-24", 95.78],
        ["IBOR FUTURE","USD-LIBOR-3M","2026-12-24", 95.86],
        ["IBOR FUTURE","USD-LIBOR-3M","2027-03-24", 95.94],
        ["IBOR FUTURE","USD-LIBOR-3M","2027-06-24", 96.02],
        ["IBOR FUTURE","USD-LIBOR-3M","2027-09-24", 96.08],
        ["IBOR FUTURE","USD-LIBOR-3M","2027-12-24", 96.14],
        ["IBOR FUTURE","USD-LIBOR-3M","2028-03-24", 96.20],
        ["IBOR FUTURE","USD-LIBOR-3M","2028-06-24", 96.25],
    ],
    columns=["DATA TYPE","DATA CONVENTION","AXIS","VALUE"],
)

MARKET_DF = pd.concat([MARKET_DF, IBOR_FUTURES_DF], ignore_index=True)

In [5]:
IBOR_SWAPS_DF = pd.DataFrame(
    [
        ["IBOR SWAP","USD-LIBOR-IRS-3M","4Y",  0.0400],
        ["IBOR SWAP","USD-LIBOR-IRS-3M","5Y",  0.0398],
        ["IBOR SWAP","USD-LIBOR-IRS-3M","7Y",  0.0402],
        ["IBOR SWAP","USD-LIBOR-IRS-3M","10Y", 0.0408],
    ],
    columns=["DATA TYPE","DATA CONVENTION","AXIS","VALUE"],
)

MARKET_DF = pd.concat([MARKET_DF, IBOR_SWAPS_DF], ignore_index=True)


In [6]:
data_objs, dc = build_yc_data_collection(MARKET_DF)

In [7]:
dc.dataMap

{('RFR FUTURE',
  'SOFR-FUTURE-3M'): Data1D(type='RFR FUTURE', conv='SOFR-FUTURE-3M', points=12),
 ('RFR SWAP',
  'USD-SOFR-OIS'): Data1D(type='RFR SWAP', conv='USD-SOFR-OIS', points=14),
 ('IBOR FUTURE',
  'USD-LIBOR-3M'): Data1D(type='IBOR FUTURE', conv='USD-LIBOR-3M', points=12),
 ('IBOR SWAP',
  'USD-LIBOR-IRS-3M'): Data1D(type='IBOR SWAP', conv='USD-LIBOR-IRS-3M', points=4)}

In [8]:
build_methods = [{
    "TARGET": "SOFR-1B",
    "REFERENCE": None,
    "INSTRUMENTS": ["SOFR-FUTURE-3M", "USD-SOFR-OIS"],
    "INTERPOLATION METHOD": "PIECEWISE_CONSTANT"
},
{
    "TARGET": "USD-LIBOR-BBA-3M",
    "REFERENCE": None,
    "INSTRUMENTS": ["USD-LIBOR-FUTURE-3M", "USD-LIBOR-IRS-3M"],
    "INTERPOLATION METHOD": "PIECEWISE_CONSTANT"
}]


In [9]:
yc = YieldCurve("2025-09-24", dc, build_methods)
print("Curve components:", yc.components.keys())

Curve components: dict_keys(['SOFR-1B', 'USD-LIBOR-BBA-3M'])


In [10]:
from fixedincomelib.date import addPeriod, moveToBusinessDay

StartDate = "2025-09-24"
print(Date(StartDate).ISO(),yc.forward(index="SOFR-1B", effectiveDate=Date(StartDate),termOrTerminationDate="3M"))
for i in range(20):
    new_date = addPeriod(StartDate,"3M")
    print(new_date.ISO(), 
          yc.forward(index="SOFR-1B", effectiveDate=new_date,termOrTerminationDate="3M"),
          yc.discountFactor(index='SOFR-1B', to_date=new_date))
    StartDate = moveToBusinessDay(input_date=new_date,biz_conv="F",hol_conv="USGS")
    

2025-09-24 0.043000000000005
2025-12-24 0.04199999999999804 0.989247430017613
2026-03-24 0.04099999999999664 0.9789682632534521
2026-06-24 0.04000000000000573 0.9688172118009167
2026-09-24 0.03919999999999328 0.9590139580079453
2026-12-24 0.03839999999999577 0.9496044331910605
2027-03-24 0.03760000000000652 0.940574914016503
2027-06-24 0.03679999999999781 0.9316230516270891
2027-09-24 0.03622521242452335 0.9229432827986367
2027-12-24 0.03681926692386772 0.9145744187699922
2028-03-24 0.035009140872462895 0.9061408817901293
2028-06-24 0.035221119046009865 0.8981078064105686
2028-09-26 0.03055554917526575 0.8899345474793607
2028-12-26 0.03055425581301474 0.8831135867099948
2029-03-26 0.030556842610468975 0.8764190040973504
2029-06-26 0.030654149545105872 0.8696281035521578
2029-09-26 0.03503374783623284 0.8628685310770882
2029-12-26 0.03503204822953965 0.8552942564634455
2030-03-26 0.03503544755279036 0.8478686129283409
2030-06-26 0.035148647109070306 0.8403445849445828
2030-09-26 0.04024

In [11]:
comp = yc.retrieveComponent("SOFR-1B")

In [12]:
yc.forwardRateGradientWrtModelParameters(index="SOFR-1B", start_time= "2025-09-24", end_time="2026-02-24", scaler =1, accumulate=False)

In [13]:
yc.gradient_

array([0.60557794, 0.41259157, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [14]:
yc.discountFactorGradientWrtModelParameters(index="SOFR-1B", to_date="2045-12-24", scaler =1, accumulate=False)

In [15]:
yc.gradient_

array([-0.10935183, -0.10815017, -0.1105535 , -0.1105535 , -0.10935183,
       -0.10815017, -0.1105535 , -0.1105535 , -0.10935183, -0.10935183,
       -0.1105535 , -0.1105535 , -0.438609  , -0.438609  , -0.438609  ,
       -0.43981067, -0.44101234, -0.43740734, -0.43740734, -2.19544836,
       -2.19544836, -0.10815017, -0.        , -0.        , -0.        ,
       -0.        ,  0.        ,  0.        ,  0.        ,  0.        ])

In [16]:
yc.discountFactorGradientWrtModelParameters(index="SOFR-1B", to_date="2045-12-24", scaler =-1, accumulate=True)

In [17]:
yc.gradient_

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [18]:
comp.state_variables


[0.042767986667723,
 0.041781031446152875,
 0.040786694242020215,
 0.03979693821161253,
 0.03900705928293265,
 0.038216851219243064,
 0.037420501824865025,
 0.036628035534984525,
 0.03603537843488787,
 0.036648982515920504,
 0.03484439937468056,
 0.035112107246030026,
 0.03043815136713759,
 0.034879532521858034,
 0.04004042935523508,
 0.039059288982859715,
 0.04243177643483296,
 0.04075982228619981,
 0.041480306901718604,
 0.041511031510561916,
 0.04462732168714627,
 0.04562955477978566,
 0.04849937096131158,
 0.04496341276106825,
 0.04939602126493097,
 0.0528503085724345]

In [19]:
comp.pillar_dates


[Date(24,12,2025),
 Date(24,3,2026),
 Date(24,6,2026),
 Date(24,9,2026),
 Date(24,12,2026),
 Date(24,3,2027),
 Date(24,6,2027),
 Date(24,9,2027),
 Date(24,12,2027),
 Date(24,3,2028),
 Date(24,6,2028),
 Date(24,9,2028),
 Date(24,9,2029),
 Date(24,9,2030),
 Date(24,9,2031),
 Date(24,9,2032),
 Date(26,9,2033),
 Date(25,9,2034),
 Date(24,9,2035),
 Date(24,9,2040),
 Date(25,9,2045),
 Date(26,9,2050),
 Date(24,9,2055),
 Date(24,9,2065),
 Date(24,9,2075),
 Date(24,9,2085)]

In [20]:
comp.pillar_times_to_date


[0.25277777777777777,
 0.5027777777777778,
 0.7583333333333333,
 1.0138888888888888,
 1.2666666666666666,
 1.5166666666666666,
 1.7722222222222221,
 2.0277777777777777,
 2.2805555555555554,
 2.533333333333333,
 2.7888888888888888,
 3.0444444444444443,
 4.058333333333334,
 5.072222222222222,
 6.086111111111111,
 7.102777777777778,
 8.122222222222222,
 9.133333333333333,
 10.144444444444444,
 15.219444444444445,
 20.294444444444444,
 25.369444444444444,
 30.43611111111111,
 40.583333333333336,
 50.727777777777774,
 60.875]

In [21]:
comp.pillar_nodes

[PillarNode(node_id='RFR FUTURE SOFR-FUTURE-3M September 24th, 2025 x December 24th, 2025', pillar_index=0, pillar_time=0.25277777777777777, pillar_date=Date(24,12,2025), start_date=Date(24,9,2025), end_date=Date(24,12,2025), instrument=<fixedincomelib.product.linear_products.ProductRfrFuture object at 0x000001E34780EFF0>, state_value=0.042767986667723),
 PillarNode(node_id='RFR FUTURE SOFR-FUTURE-3M December 24th, 2025 x March 24th, 2026', pillar_index=1, pillar_time=0.5027777777777778, pillar_date=Date(24,3,2026), start_date=Date(24,12,2025), end_date=Date(24,3,2026), instrument=<fixedincomelib.product.linear_products.ProductRfrFuture object at 0x000001E34A0816D0>, state_value=0.041781031446152875),
 PillarNode(node_id='RFR FUTURE SOFR-FUTURE-3M March 24th, 2026 x June 24th, 2026', pillar_index=2, pillar_time=0.7583333333333333, pillar_date=Date(24,6,2026), start_date=Date(24,3,2026), end_date=Date(24,6,2026), instrument=<fixedincomelib.product.linear_products.ProductRfrFuture object

In [22]:
yc.jacobian()

array([[-1.00000000e+02,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -9.89247430e+01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+

In [23]:
yc._jacobian_row_labels

['RFR FUTURE SOFR-FUTURE-3M September 24th, 2025 x December 24th, 2025',
 'RFR FUTURE SOFR-FUTURE-3M December 24th, 2025 x March 24th, 2026',
 'RFR FUTURE SOFR-FUTURE-3M March 24th, 2026 x June 24th, 2026',
 'RFR FUTURE SOFR-FUTURE-3M June 24th, 2026 x September 24th, 2026',
 'RFR FUTURE SOFR-FUTURE-3M September 24th, 2026 x December 24th, 2026',
 'RFR FUTURE SOFR-FUTURE-3M December 24th, 2026 x March 24th, 2027',
 'RFR FUTURE SOFR-FUTURE-3M March 24th, 2027 x June 24th, 2027',
 'RFR FUTURE SOFR-FUTURE-3M June 24th, 2027 x September 24th, 2027',
 'RFR FUTURE SOFR-FUTURE-3M September 24th, 2027 x December 24th, 2027',
 'RFR FUTURE SOFR-FUTURE-3M December 24th, 2027 x March 24th, 2028',
 'RFR FUTURE SOFR-FUTURE-3M March 24th, 2028 x June 24th, 2028',
 'RFR FUTURE SOFR-FUTURE-3M June 24th, 2028 x September 24th, 2028',
 'RFR SWAP USD-SOFR-OIS September 24th, 2029',
 'RFR SWAP USD-SOFR-OIS September 24th, 2030',
 'RFR SWAP USD-SOFR-OIS September 24th, 2031',
 'RFR SWAP USD-SOFR-OIS Septemb


## 4. Value Bullet Cashflow


In [24]:
# Bullet cashflow PV
bullet = ProductBulletCashflow("2028-09-24", "USD", 1000000, "LONG")
ve = ValuationEngineRegistry().new_valuation_engine(
    yc,
    {"FUNDING INDEX": "SOFR-1B"},
    bullet
)
ve.calculateValue()
print("Bullet CF PV:", ve.value)

Bullet CF PV: ['USD', np.float64(890085.0488843094)]


## 5. Value IBOR Cashflow

In [40]:
ibor_cf = ProductIborCashflow(
    startDate="2025-09-29",
    endDate="2025-12-29",
    index="USD-LIBOR-BBA-3M",
    spread=0.0,
    notional=1000000,
    longOrShort="SHORT"
)
ve = ValuationEngineRegistry().new_valuation_engine(yc, {"FUNDING INDEX": "SOFR-1B"}, ibor_cf)
ve.calculateValue()
print("Ibor CF PV:", ve.value)

Ibor CF PV: ['USD', -10084.91996765759]


## 6. Value OIS Cashflow

In [ ]:
ois_cf = ProductOvernightIndexCashflow(
    effectiveDate="2025-09-24",
    termOrEnd="6M",
    index="SOFR-1B",
    compounding="COMPOUND",
    spread=0.0,
    notional=1000000,
    longOrShort="LONG"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc,
    {"FUNDING INDEX": "SOFR-1B"},
    ois_cf
)
ve.calculateValue()
print("OIS CF PV:", ve.value)

OIS CF PV: ['USD', 21483.57361111164]


## 7. Value Futures

In [42]:
# IBOR Future PV
future = ProductFuture(
    effectiveDate="2025-09-29",
    index="USD-LIBOR-BBA-3M",
    strike=99.5,
    notional=1_000_000,
    longOrShort="LONG"
)
ve = ValuationEngineRegistry().new_valuation_engine(yc, {"FUNDING INDEX": "SOFR-1B"}, future)
ve.calculateValue()
print("Future PV:", ve.value)

Future PV: ['USD', np.float64(-3450113.4299419164)]


In [28]:
# RFR Future PV
rfr_future = ProductRfrFuture(
    effectiveDate="2025-09-24",
    termOrEnd="3M",
    index="SOFR-1B",
    compounding="AVERAGE",
    strike=95.9,
    notional=1000000,
    longOrShort="SHORT"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc,
    {"FUNDING INDEX": "SOFR-1B"},
    rfr_future
)
ve.calculateValue()
print("RFR Future PV:", ve.value)

RFR Future PV: ['USD', np.float64(175392.01143027726)]


In [29]:
ve.calculateFirstOrderRisk()
print(ve.firstOrderRisk_)

[99955664.79711068        0.                0.                0.
        0.                0.                0.                0.
        0.                0.                0.                0.
        0.                0.                0.                0.
        0.                0.                0.                0.
        0.                0.                0.                0.
        0.                0.                0.                0.
        0.                0.        ]


## 8. Value a Portfolio

In [30]:
# Simple portfolio of bullet + IBOR CF
portfolio = ProductPortfolio([bullet, ois_cf], weights=[0.5, 0.5])
ve = ValuationEngineRegistry().new_valuation_engine(
    yc,
    {"FUNDING INDEX": "SOFR-1B"},
    portfolio
)
ve.calculateValue()
print("Portfolio PV:", ve.value)

Portfolio PV: ['USD', np.float64(911568.622495421)]


## 9. Value Swaps (IBOR & OIS)

In [47]:
# IBOR Swap
ibor_swap = ProductIborSwap(
    effectiveDate="2025-09-27",
    maturityDate="2026-09-27",
    frequency="6M",
    iborIndex="USD-LIBOR-BBA-3M",
    spread=0.0,
    fixedRate=0.015,
    notional=1_000_000,
    position="SHORT"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc, {"FUNDING INDEX": "SOFR-1B"}, ibor_swap
)
ve.calculateValue()
print("IBOR Swap PV:", ve.value, "Par Rate:", ve.parRateOrSpread())



IBOR Swap PV: ['USD', np.float64(5371.148217124453)] Par Rate: 0.020485171638684553


In [48]:
ve.calculateFirstOrderRisk()
ve.firstOrderRisk_

array([-1.35770691e+03,  1.01140448e+03, -6.29033250e+02,  1.74823008e+03,
        8.21021778e+01,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  5.07898480e+05,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00])

In [32]:
# OIS Swap
ois_swap = ProductOvernightSwap(
    effectiveDate="2025-09-24",
    maturityDate="2045-09-24",
    frequency="6M",
    overnightIndex="SOFR-1B",
    spread=0.0,
    fixedRate=0.045,
    notional=1000000,
    position="LONG"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc, {"FUNDING INDEX": "SOFR-1B"}, ois_swap
)
ve.calculateValue()
print("OIS Swap PV:", ve.value, "Par Rate:", ve.parRateOrSpread())

OIS Swap PV: ['USD', np.float64(62491.06009689486)] Par Rate: 0.0405001202500912


In [33]:
ve.calculateFirstOrderRisk()
ve.firstOrderRisk_

array([ -268574.12908005,  -265622.76502422,  -265865.15302487,
        -265865.15302487,  -257399.71381028,  -254571.14552665,
        -254789.93019523,  -254789.93019523,  -246654.5847353 ,
        -246654.5847353 ,  -244096.87227304,  -244096.87227304,
        -937537.61309711,  -897632.06580721,  -859117.50779538,
        -824334.68183579,  -790719.90258028,  -750163.13807317,
        -717660.50061755, -3156522.81265421, -2519632.47125135,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ])

In [34]:
from fixedincomelib.utilities import createValueReport

pv_only = createValueReport({"FUNDING INDEX": "SOFR-1B"}, yc, ois_swap, request="value")

In [35]:
print(pv_only)

62491.06009689486
